In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from pandas import read_csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from pandas import get_dummies
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression


train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [3]:
# Checking Nulls
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
# Checking Nulls
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
# Filling Null Values in Train data
train_data['Embarked'] = train_data['Embarked'].replace(np.nan, 'Q')
train_data["Age"].fillna(train_data["Age"].mean(),inplace=True)

In [6]:
# Filling Null Values in Test data
test_data['Fare'] = test_data['Fare'].replace(np.nan, 35)
test_data["Age"].fillna(test_data["Age"].mean(),inplace=True)

In [7]:
# Dropping unwanted colums
df_train = train_data.drop(["PassengerId","Name","Cabin","Ticket"],axis = 1)
df_test = test_data.drop(["PassengerId","Name","Cabin","Ticket"],axis = 1)

In [8]:
# Creating a new column with total Family count

df_train["Family_Size"]=df_train["SibSp"]+df_train["Parch"]
df_test["Family_Size"]=df_test["SibSp"]+df_test["Parch"]

In [9]:
# Checking Null values in Train data
df_train.isnull().sum()

Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Family_Size    0
dtype: int64

In [10]:
# Checking Null values in Test data
df_test.isnull().sum()

Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Family_Size    0
dtype: int64

In [11]:
# Converting Both Train and test colums to Categorial data to use df.dummies()
df_train["Sex"] = df_train["Sex"].astype('category')
df_train["Embarked"] = df_train["Embarked"].astype('category')
df_test["Sex"] = df_test["Sex"].astype('category')
df_test["Embarked"] = df_test["Embarked"].astype('category')

In [12]:
# Using df.dummies to one hot encode the Sex and Embarked columns

df_train = pd.get_dummies(df_train, columns=["Sex","Embarked"])
df_test = pd.get_dummies(df_test, columns=["Sex","Embarked"])

In [13]:
# Scaling values in Fare and Age Column column
mms = MinMaxScaler()
df_train[['Fare']] = mms.fit_transform(df_train[['Fare']])
df_test[['Fare']] = mms.fit_transform(df_test[['Fare']])

df_train[['Age']] = mms.fit_transform(df_train[['Age']])
df_test[['Age']] = mms.fit_transform(df_test[['Age']])

In [14]:
# Shape of Train and Test dataset
print(df_train.shape)
print(df_test.shape)

(891, 12)
(418, 11)


In [15]:
# Lets train Models based on df_train to find out Model with best settings
# First we need to Divide the data into data and output set from df_train

train_input = df_train.drop(columns=['Survived'])
train_result = df_train["Survived"]

In [16]:
# Splitting the data using train_test_split

x_train,x_test,y_train,y_test=train_test_split(train_input,train_result,test_size=0.10,random_state=40)

In [17]:
model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=150, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

model.fit(x_train,y_train)

print(model.score(x_test,y_test))

0.8333333333333334


In [18]:
# Predicting the values for test data
result = model.predict(df_test)
print(result)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [19]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived':result})
output.to_csv('my_submission.csv', index=False)